In [5]:
import os
import commands
def print_list(x):
    for i in x:
        print i
def orden(x):
    print x
    #os.system(x)


#  Funciones de preprocesamiento

In [6]:
############# T1_Preproc
def T1_Preproc(fol_in,fol_out):
    iin=fol_in+'highres.nii.gz'
    iout=fol_out+'highres_reorient.nii.gz'
    comm='fsl5.0-fslreorient2std '+iin+' '+iout+' ; '
    
    #-----------------------------------------
    iin=fol_out+'highres_reorient.nii.gz'
    iout=fol_out+'highres_bet.nii.gz'
    comm=comm+'fsl5.0-bet '+iin+' '+iout+' -m -f 0.3 -g -0.05 ; rm '+iin+' ; '
    
    #------------------------------------------  
    iin=fol_out+'highres_bet.nii.gz'
    iout=fol_out+'highres_bet_5_mm.nii.gz'
    comm=comm+'fsl5.0-fslmaths '+iin+' -kernel gauss 5 -fmean '+iout+' ; rm '+iin
    return comm, iout

In [7]:
############# T2_Preproc
def T2_Preproc(fol_in,fol_out):
    iin=fol_in+'rsfMRI.nii.gz'
    iout_med=fol_out+'rsfMRI_med.nii.gz'
    iout=fol_out+'rsfMRI_01.nii.gz'
    
    comm='fsl5.0-fslmaths '+iin+' -Tmean '+iout_med+' ; '
    comm=comm+'fsl5.0-fslmaths '+iin+' -bptf 20 -1 -add '+iout_med+' '+iout
    comm=comm+' ; rm '+iout_med+' ; rm '+fol_out+'highres_bet_mask.nii.gz'
    return comm, iout

############# T2_Preproc_MCflirt
def T2_Preproc_MC(fol_in,fol_out):
    iin0=fol_in+'rsfMRI.nii.gz'
    iin=fol_out+'rsfMRI_MC.nii.gz'
    iout_med=fol_out+'rsfMRI_med.nii.gz'
    iout=fol_out+'rsfMRI_MC_01.nii.gz'
    #-----------------------------------------
    comm='fsl5.0-mcflirt -in '+iin0+'  -out '+iin+' -refvol 0 -plots -report ; '
    #-----------------------------------------
    comm=comm+'fsl5.0-fslmaths '+iin+' -Tmean '+iout_med+' ; '
    comm=comm+'fsl5.0-fslmaths '+iin+' -bptf 20 -1 -add '+iout_med+' '+iout+' ; '
    comm=comm+'rm '+iout_med+' ; rm '+iin
    
    return comm, iout

# Funciones de filtro

In [8]:
def Gauss(iin,mm,iout):
    comm='fsl5.0-fslmaths '+iin+' -kernel gauss '+mm+' -fmean '+iout
    return comm

def Aniso(iin,it,iout,fol_out,j):
    place=1
    comm=''
    if(place==1):
        ##usado para servidor
        comm=comm+'(cd ~/abin/; ./3danisosmooth -3D -iters '+it+' -prefix '+iout+' '+iin+')' 
        comm=comm+' ; (cd ~/abin/; ./3dAFNItoNIFTI -prefix '+iout+' '+iout+'+orig)'
        comm=comm+' ; rm -R '+fol_out+str(j).zfill(4)+'+orig.BRIK'
        comm=comm+' ; rm -R '+fol_out+str(j).zfill(4)+'+orig.HEAD'     
    else:
        ##usado por computador 
        comm=comm+'3danisosmooth -3D -iters '+it+' -prefix '+iout+' '+iin
        comm=comm+' ; 3dAFNItoNIFTI -prefix '+iout+' '+iout+'+tlrc'
        comm=comm+' ; rm -R '+fol_out+str(j).zfill(4)+'+tlrc.BRIK'
        comm=comm+' ; rm -R '+fol_out+str(j).zfill(4)+'+tlrc.HEAD'
    return comm

In [9]:
# genera comandos
#[ indicativo_experimento , 1 graduente 0 curvatura , iteraciones, delta_tiempo , conductancia o radio ]

def AnisoITK(typ,iin,iout,niter,dt,conductancia):
    type_filter=['AnisotropicCurvature','AnisotropicGradient'][typ]
    comm='( cd /home/users/aforero/itk_lib/InsightToolkit-4.10.1/Examples/ ; ./'+type_filter+' '+iin+' '+iout+' '+str(niter)+' '+str(dt)+' '+str(conductancia)+' )'
    return comm

# Crear carpeta

In [23]:
def newFol(fol_out):
    comm='mkdir '+fol_out
    return comm

#  Separa Une

In [10]:
def Split(iin,fol_out):
    comm='mkdir '+fol_out+' ; '
    iout=fol_out
    comm=comm+'fsl5.0-fslsplit '+iin+' '+iout+' -t'
    return comm
    

In [11]:
def merge(fol_in,iout):
    comm='fsl5.0-fslmerge -t '+iout
    for i in range(120):
        comm=comm+' '+fol_in+str(i).zfill(4)+'.nii.gz'
    return comm

# Registro

In [12]:
def Reg(iin,iout,iref,ioutmat,dof):
    comm='fsl5.0-flirt -dof '+dof+' -in '+iin+' -ref '+iref+' -omat '+ioutmat+' -out '+iout
    return comm
    

#  Normalización

In [13]:
def Norm (iin, iout,iatlas, mat_ref, iref):
    comm='fsl5.0-flirt  -in '+iin+' -ref'+iatlas+' -applyxfm   -init '+ mat_ref+'  -out '+iout
    return comm

# ICA

In [14]:
Data0='''
# FEAT version number
set fmri(version) 3.14

# Are we in MELODIC?
set fmri(inmelodic) 1

# Analysis level
# 1 : First-level analysis
# 2 : Higher-level analysis
set fmri(level) 1

# Which stages to run
# 0 : No first-level analysis (registration and/or group stats only)
# 7 : Full first-level analysis
# 1 : Pre-processing
# 2 : Statistics
set fmri(analysis) 7

# Use relative filenames
set fmri(relative_yn) 0

# Balloon help
set fmri(help_yn) 1

# Run Featwatcher
set fmri(featwatcher_yn) 1

# Cleanup first-level standard-space images
set fmri(sscleanup_yn) 0

# Output directory
set fmri(outputdir) "/home/arbey/Escritorio/Tesis/icA"

# TR(s)
set fmri(tr) 2.5

# Total volumes
set fmri(npts) 120

# Delete volumes
set fmri(ndelete) 0

# Perfusion tag/control order
set fmri(tagfirst) 1

# Number of first-level analyses
set fmri(multiple) 1

# Higher-level input type
# 1 : Inputs are lower-level FEAT directories
# 2 : Inputs are cope images from FEAT directories
set fmri(inputtype) 2

# Carry out pre-stats processing?
set fmri(filtering_yn) 1

# Brain/background threshold, %
set fmri(brain_thresh) 10

# Critical z for design efficiency calculation
set fmri(critical_z) 5.3

# Noise level
set fmri(noise) 0.66

# Noise AR(1)
set fmri(noisear) 0.34

# Motion correction
# 0 : None
# 1 : MCFLIRT
set fmri(mc) 0

# Spin-history (currently obsolete)
set fmri(sh_yn) 0

# B0 fieldmap unwarping?
set fmri(regunwarp_yn) 0

# EPI dwell time (ms)
set fmri(dwell) 0.7

# EPI TE (ms)
set fmri(te) 35

# % Signal loss threshold
set fmri(signallossthresh) 10

# Unwarp direction
set fmri(unwarp_dir) y-

# Slice timing correction
# 0 : None
# 1 : Regular up (0, 1, 2, 3, ...)
# 2 : Regular down
# 3 : Use slice order file
# 4 : Use slice timings file
# 5 : Interleaved (0, 2, 4 ... 1, 3, 5 ... )
set fmri(st) 0

# Slice timings file
set fmri(st_file) ""

# BET brain extraction
set fmri(bet_yn) 0

# Spatial smoothing FWHM (mm)
set fmri(smooth) 0.0

# Intensity normalization
set fmri(norm_yn) 0

# Perfusion subtraction
set fmri(perfsub_yn) 0

# Highpass temporal filtering
set fmri(temphp_yn) 0

# Lowpass temporal filtering
set fmri(templp_yn) 0

# MELODIC ICA data exploration
set fmri(melodic_yn) 0

# Carry out main stats?
set fmri(stats_yn) 1

# Carry out prewhitening?
set fmri(prewhiten_yn) 1

# Add motion parameters to model
# 0 : No
# 1 : Yes
set fmri(motionevs) 0
set fmri(motionevsbeta) ""
set fmri(scriptevsbeta) ""

# Robust outlier detection in FLAME?
set fmri(robust_yn) 0

# Higher-level modelling
# 3 : Fixed effects
# 0 : Mixed Effects: Simple OLS
# 2 : Mixed Effects: FLAME 1
# 1 : Mixed Effects: FLAME 1+2
set fmri(mixed_yn) 2

# Number of EVs
set fmri(evs_orig) 1
set fmri(evs_real) 1
set fmri(evs_vox) 0

# Number of contrasts
set fmri(ncon_orig) 1
set fmri(ncon_real) 1

# Number of F-tests
set fmri(nftests_orig) 0
set fmri(nftests_real) 0

# Add constant column to design matrix? (obsolete)
set fmri(constcol) 0

# Carry out post-stats steps?
set fmri(poststats_yn) 1

# Pre-threshold masking?
set fmri(threshmask) ""

# Thresholding
# 0 : None
# 1 : Uncorrected
# 2 : Voxel
# 3 : Cluster
set fmri(thresh) 3

# P threshold
set fmri(prob_thresh) 0.05

# Z threshold
set fmri(z_thresh) 2.3

# Z min/max for colour rendering
# 0 : Use actual Z min/max
# 1 : Use preset Z min/max
set fmri(zdisplay) 0

# Z min in colour rendering
set fmri(zmin) 2

# Z max in colour rendering
set fmri(zmax) 8

# Colour rendering type
# 0 : Solid blobs
# 1 : Transparent blobs
set fmri(rendertype) 1

# Background image for higher-level stats overlays
# 1 : Mean highres
# 2 : First highres
# 3 : Mean functional
# 4 : First functional
# 5 : Standard space template
set fmri(bgimage) 1

# Create time series plots
set fmri(tsplot_yn) 1

# Registration to initial structural
set fmri(reginitial_highres_yn) 0

# Search space for registration to initial structural
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(reginitial_highres_search) 90

# Degrees of Freedom for registration to initial structural
set fmri(reginitial_highres_dof) 3

# Registration to main structural
set fmri(reghighres_yn) 0

# Search space for registration to main structural
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(reghighres_search) 90

# Degrees of Freedom for registration to main structural
set fmri(reghighres_dof) BBR

# Registration to standard image?
set fmri(regstandard_yn) 0

# Use alternate reference images?
set fmri(alternateReference_yn) 0

# Standard image
set fmri(regstandard) "/usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain"

# Search space for registration to standard space
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(regstandard_search) 90

# Degrees of Freedom for registration to standard space
set fmri(regstandard_dof) 12

# Do nonlinear registration from structural to standard space?
set fmri(regstandard_nonlinear_yn) 0

# Control nonlinear warp field resolution
set fmri(regstandard_nonlinear_warpres) 10 

# High pass filter cutoff
set fmri(paradigm_hp) 1.0

# Total voxels
set fmri(totalVoxels) 108315480


# Number of lower-level copes feeding into higher-level analysis
set fmri(ncopeinputs) 0

# 4D AVW data or FEAT directory (1)
set feat_files(1) "/home/users/aforero/Tesis/Exp_C/ImagesC/C001/Baseline/rsfMRI_Baseline"

# Add confound EVs text file
set fmri(confoundevs) 0

# Resampling resolution
set fmri(regstandard_res) 4

# Variance-normalise timecourses
set fmri(varnorm) 1

# Automatic dimensionality estimation
set fmri(dim_yn) 0

# Output components
set fmri(dim) 30

# 1 : Single-session ICA
# 2 : Multi-session temporal concatenation
# 3 : Multi-session tensor TICA
set fmri(icaopt) 1

# Threshold IC maps
set fmri(thresh_yn) 1

# Mixture model threshold
set fmri(mmthresh) 0.5

# Output full stats folder
set fmri(ostats) 0

# Timeseries and subject models
set fmri(ts_model_mat) ""
set fmri(ts_model_con) ""
set fmri(subject_model_mat) ""
set fmri(subject_model_con) ""


##########################################################
# Now options that don't appear in the GUI

# Alternative (to BETting) mask image
set fmri(alternative_mask) ""

# Initial structural space registration initialisation transform
set fmri(init_initial_highres) ""

# Structural space registration initialisation transform
set fmri(init_highres) ""

# Standard space registration initialisation transform
set fmri(init_standard) ""

# For full FEAT analysis: overwrite existing .feat output dir?
set fmri(overwrite_yn) 0
'''

In [15]:
def ICASingle(Data,iin,fol_out,Name):
    Name=Name.replace('.nii.gz','')
    
    r1='''set feat_files(1) "/home/users/aforero/Tesis/Exp_C/ImagesC/C001/Baseline/rsfMRI_Baseline"'''
    r2='''set feat_files(1) "'''+iin+'''"'''
    Data=Data.replace(r1, r2)
    
    r1='''set fmri(outputdir) "/home/arbey/Escritorio/Tesis/icA"'''
    r2='''set fmri(outputdir) "'''+fol_out+Name+'''"'''
    Data=Data.replace(r1, r2)

    
    melodic_conf='feat_'+Name+'.fsf'
    melodic_out=fol_out+'feat_'+Name+'.fsf'
    
    fil_tem=open(melodic_conf,'w')
    fil_tem.write(Data)
    fil_tem.close()
    
    melodic_out=Dir_out+melodic_conf
    os.system('mv '+melodic_conf+' '+melodic_out)
    os.system('rm '+melodic_conf)
    comm='fsl5.0-feat '+melodic_out
    return comm

# Seleccion DMN

In [16]:
blablabla

NameError: name 'blablabla' is not defined

# Test

In [ ]:
PSNR

# Graficas

In [ ]:
DMN HD

Medias, psnr

# Main


In [ ]:


T1_Preproc(fol_in,fol_out)
T2_Preproc(fol_in,fol_out)
Split(iin,fol_out)

Filtro  i.e  Aniso(iin,it,iout,fol_out,j)

Reg(iin,iout,iref,ioutmat,dof)
Reg(iin,iout,iref,ioutmat,dof)
Norm (iin, iout,iatlas, mat_ref, iref)
  
merge(fol_in,iout)

ICASingle(Data0,Dir_out,Name,iin)

In [26]:
def TestIn(ID,testName):
    imgID='C'+str(ID).zfill(3)
    commFolder=newFol('/home/arbey/Escritorio/CPrueba/'+testName)
    fol_in='/home/arbey/Escritorio/CPrueba/ImagesC/'+imgID+'/'
    fol_out='/home/arbey/Escritorio/CPrueba/'+testName+'/'+imgID+'/'

    comm,iiout=T1_Preproc(fol_in,fol_out)
    print commFolder
    print comm

In [27]:
TestIn(1,'filtro_1CN')

mkdir /home/arbey/Escritorio/CPrueba/filtro_1CN
fsl5.0-fslreorient2std /home/arbey/Escritorio/CPrueba/ImagesC/C001/highres.nii.gz /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_reorient.nii.gz ; fsl5.0-bet /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_reorient.nii.gz /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_bet.nii.gz -m -f 0.3 -g -0.05 ; rm /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_reorient.nii.gz ; fsl5.0-fslmaths /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_bet.nii.gz -kernel gauss 5 -fmean /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_bet_5_mm.nii.gz ; rm /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_bet.nii.gz


In [127]:
def orden(x):
    print x
    #os.system(x)

In [128]:
class Simulacion:
    def __init__(self,ID,folder):
        self._ID=ID
        self._folder=folder
        
    
    def FolderInit(self,fol_in):
        print
        print "FolderInit()"
        hr_iin=fol_in+'highres.nii.gz'
        rs_iin=fol_in+'rsfMRI.nii.gz'
        orden('mkdir '+self._folder+self._ID)
        return hr_iin, rs_iin

    def Preproc_T1(self,iin):
        print
        print  "Preproc_T1()"
        orden('mkdir '+self._folder+self._ID+'/Preproc_T1')
        pass

    def Noise_T2(self,img4D,NoiseConfig):
        print
        print "Noise_T2()"
        orden('mkdir '+self._folder+self._ID+'/Noise_T2')
        if(NoiseConfig[0]=='Directo'):
            return img4D
        elif(NoiseConfig[0]=='Noise'):
            print 'Possum'
            ###llama funcion para adicionar ruido y env[ia configuracion
            img4D=adicionaRuido()
            return img4D

    def Preproc_T2(self,iin):
        print
        print "Preproc_T2()"
        orden('mkdir '+self._folder+self._ID+'/Preproc_T2')
        pass

    def MotCorr(self,list_iin,MCConfig):
        print
        print "MotCorr()"
        orden('mkdir '+self._folder+self._ID+'/MotCorr')
        if(MCConfig[0]=='Directo'):
            print "Directo"
            return img4D
        elif(MCConfig[0]=='Mcflirt'):
            print "Mcflirt"
            return img4D
        elif(MCConfig[0]=='Gauss'):
            print 'Gauss'
            return img4D
        elif(MCConfig[0]=='AnisoAFNI'):
            print 'AnisoAFNI'
            return img4D
        elif(MCConfig[0]=='AnisoGrad'):
            print 'AnisoGrad'
            return img4D
        elif(MCConfig[0]=='AnisoCurv'):
            print 'AnisoCurv'
            return img4D
        pass

    def Reg(self,iin_Ref,list_Reg):
        print
        print "Reg()"
        orden('mkdir '+self._folder+self._ID+'/Reg')
        pass

    def Norm(self,list_Norm):
        print
        print "Norm()"
        orden('mkdir '+self._folder+self._ID+'/Norm')
        pass

    def Ica(self,desc_4D):
        print
        print "Ica()"
        orden('mkdir '+self._folder+self._ID+'/Ica')
        pass

    def SelectDMN(self,IC_4D):
        print
        print "SelectDMN()"
        orden('mkdir '+self._folder+self._ID+'/SelectDMN')
        pass


In [129]:
def Prueba(IDExperimento,Folder,MCConfig,NoiseConfig,Descripcion):
    sim=Simulacion(IDExperimento,Folder)
    
    hr_iin, rs_iin=sim.FolderInit(Folder)
    
    hr_5mm=sim.Preproc_T1(hr_iin)
    
    imgN=sim.Noise_T2(rs_iin,NoiseConfig)
    
    NameOutP2=sim.Preproc_T2(imgN)
    
    list_MotCorr=sim.MotCorr(NameOutP2,MCConfig)
    
    list_AnatReg=sim.Reg(hr_5mm,list_MotCorr)
    
    AtlasReg=sim.Norm(list_AnatReg)
    
    IC=sim.Ica(AtlasReg)
    
    DMN=sim.SelectDMN(IC) 
    

In [130]:
Prueba('mcflirt_001', 
       '/home/arbey/Escritorio/CPrueba/ImagesC/C001/', 
       ['MCFlirt',0,0,0], 
       ['Directo',0,0,0], 
       "Experimento prueba")


FolderInit()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001

Preproc_T1()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1

Noise_T2()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Noise_T2

Preproc_T2()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T2

MotCorr()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/MotCorr

Reg()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Reg

Norm()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Norm

Ica()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Ica

SelectDMN()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/SelectDMN


In [131]:
t=['pp',5,3.5]
print t[0]
if (t[0]=='pp'):
    print 'config1'
    

pp
config1
